### MobileNet

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions

In [2]:
def _conv_block(inputs, filters, kernel=(3, 3), strides=(1, 1)):
    x = layers.Conv2D(filters, kernel,
               padding='same',
               use_bias=False,
               strides=strides,
               name='conv1')(inputs)
    x = layers.BatchNormalization(name='conv1_bn')(x)
    return layers.Activation(tf.nn.relu6, name='conv1_relu')(x)


def _depthwise_conv_block(inputs, pointwise_conv_filters,
                          depth_multiplier=1, strides=(1, 1), block_id=1):

    x = layers.DepthwiseConv2D((3, 3),
                        padding='same',
                        depth_multiplier=depth_multiplier,
                        strides=strides,
                        use_bias=False,
                        name='conv_dw_%d' % block_id)(inputs)

    x = layers.BatchNormalization(name='conv_dw_%d_bn' % block_id)(x)
    x = layers.Activation(tf.nn.relu6, name='conv_dw_%d_relu' % block_id)(x)

    x = layers.Conv2D(pointwise_conv_filters, (1, 1),
               padding='same',
               use_bias=False,
               strides=(1, 1),
               name='conv_pw_%d' % block_id)(x)
    x = layers.BatchNormalization(name='conv_pw_%d_bn' % block_id)(x)
    return layers.Activation(tf.nn.relu6, name='conv_pw_%d_relu' % block_id)(x)

# def relu6(x):
#     return K.relu(x, max_value=6)


In [3]:
def MobileNet(input_shape=[224,224,3],
              depth_multiplier=1,
              dropout=1e-3,
              classes=1000):


    img_input = layers.Input(shape=input_shape)

    # 224,224,3 -> 112,112,32  
    x = _conv_block(img_input, 32, strides=(2, 2))
    # 112,112,32 -> 112,112,64
    x = _depthwise_conv_block(x, 64, depth_multiplier, block_id=1)


    # 112,112,64 -> 56,56,128
    x = _depthwise_conv_block(x, 128, depth_multiplier,
                              strides=(2, 2), block_id=2)
    # 56,56,128 -> 56,56,128
    x = _depthwise_conv_block(x, 128, depth_multiplier, block_id=3)


    # 56,56,128 -> 28,28,256
    x = _depthwise_conv_block(x, 256, depth_multiplier,
                              strides=(2, 2), block_id=4)
    # 28,28,256 -> 28,28,256
    x = _depthwise_conv_block(x, 256, depth_multiplier, block_id=5)
    

    # 28,28,256 -> 14,14,512
    x = _depthwise_conv_block(x, 512, depth_multiplier,
                              strides=(2, 2), block_id=6)
    # 14,14,512 -> 14,14,512
    x = _depthwise_conv_block(x, 512, depth_multiplier, block_id=7)
    x = _depthwise_conv_block(x, 512, depth_multiplier, block_id=8)
    x = _depthwise_conv_block(x, 512, depth_multiplier, block_id=9)
    x = _depthwise_conv_block(x, 512, depth_multiplier, block_id=10)
    x = _depthwise_conv_block(x, 512, depth_multiplier, block_id=11)

    # 14,14,512 -> 7,7,1024
    x = _depthwise_conv_block(x, 1024, depth_multiplier,
                              strides=(2, 2), block_id=12)
    x = _depthwise_conv_block(x, 1024, depth_multiplier, block_id=13)

    # 7,7,1024 -> 1,1,1024
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Reshape((1, 1, 1024), name='reshape_1')(x)
    x = layers.Dropout(dropout, name='dropout')(x)

    x = layers.Conv2D(classes, (1, 1),padding='same', name='conv_preds')(x)
    x = layers.Activation('softmax', name='act_softmax')(x)
    x = layers.Reshape((classes,), name='reshape_2')(x)

    inputs = img_input

    model = keras.Model(inputs, x, name='mobilenet_1_0_224_tf')
    
    return model



In [4]:
model = MobileNet(input_shape=(224, 224, 3))

model.summary()

Model: "mobilenet_1_0_224_tf"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 

In [5]:
BASE_WEIGHT_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.6/'

model_name = 'mobilenet_1_0_224_tf.h5'

weigh_path = BASE_WEIGHT_PATH + model_name
weights_path = tf.keras.utils.get_file(model_name,
                        weigh_path,
                        cache_subdir='models')

model.load_weights(weights_path)


In [6]:
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

In [7]:
img_path = 'image_of_predict/Koalas.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img

x = image.img_to_array(img)
x.shape

(224, 224, 3)

In [8]:
x = np.expand_dims(x, axis=0)
x.shape

(1, 224, 224, 3)

In [9]:
np.max(x),np.min(x),np.mean(x)

(255.0, 0.0, 126.69101)

In [10]:
x = preprocess_input(x)

In [11]:
np.max(x),np.min(x),np.mean(x)

(1.0, -1.0, -0.0063449624)

In [12]:
preds = model.predict(x)
preds

array([[1.66683101e-09, 1.78242462e-10, 1.54579682e-10, 4.71238257e-11,
        5.47824806e-13, 1.34631500e-10, 1.80035067e-12, 3.33023470e-11,
        8.66250544e-11, 1.46152274e-06, 2.54352817e-10, 5.35491096e-12,
        6.37449052e-11, 1.14762677e-09, 3.67496145e-10, 2.61130617e-10,
        1.42290901e-10, 5.23074743e-11, 1.34893152e-10, 3.20633485e-11,
        1.26672062e-10, 7.74048742e-11, 5.53237013e-12, 1.29899369e-09,
        2.95086782e-08, 6.76697309e-11, 3.74290909e-09, 7.22181914e-09,
        2.15110263e-09, 2.24646215e-07, 1.34276439e-08, 1.11582096e-07,
        2.27703467e-07, 4.62301086e-10, 1.36983258e-09, 2.56681870e-10,
        5.82845150e-09, 8.90768792e-10, 2.37257614e-09, 5.15947178e-08,
        3.38790357e-10, 4.38484166e-10, 9.86231374e-11, 5.60639961e-08,
        1.73917325e-10, 1.26324867e-10, 4.03473921e-10, 1.83898230e-08,
        2.32428676e-09, 5.86946935e-10, 3.11995672e-11, 9.47893497e-09,
        4.71897688e-10, 7.39164771e-11, 1.78983092e-10, 4.747736

In [13]:
np.argmax(preds)

105

In [14]:
preds

array([[1.66683101e-09, 1.78242462e-10, 1.54579682e-10, 4.71238257e-11,
        5.47824806e-13, 1.34631500e-10, 1.80035067e-12, 3.33023470e-11,
        8.66250544e-11, 1.46152274e-06, 2.54352817e-10, 5.35491096e-12,
        6.37449052e-11, 1.14762677e-09, 3.67496145e-10, 2.61130617e-10,
        1.42290901e-10, 5.23074743e-11, 1.34893152e-10, 3.20633485e-11,
        1.26672062e-10, 7.74048742e-11, 5.53237013e-12, 1.29899369e-09,
        2.95086782e-08, 6.76697309e-11, 3.74290909e-09, 7.22181914e-09,
        2.15110263e-09, 2.24646215e-07, 1.34276439e-08, 1.11582096e-07,
        2.27703467e-07, 4.62301086e-10, 1.36983258e-09, 2.56681870e-10,
        5.82845150e-09, 8.90768792e-10, 2.37257614e-09, 5.15947178e-08,
        3.38790357e-10, 4.38484166e-10, 9.86231374e-11, 5.60639961e-08,
        1.73917325e-10, 1.26324867e-10, 4.03473921e-10, 1.83898230e-08,
        2.32428676e-09, 5.86946935e-10, 3.11995672e-11, 9.47893497e-09,
        4.71897688e-10, 7.39164771e-11, 1.78983092e-10, 4.747736

In [15]:
preds
print('Predicted:', decode_predictions(preds, 1))

Predicted: [[('n01882714', 'koala', 0.99852633)]]
